In [20]:
from collections import Counter
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# This is an INDIVIDUAL Assignment. Collaboration with other students is strictly prohibited

## Building a Decision Tree from Scratch

### **Node Representation [3 points]**

- **Purpose**: Represents a node in the decision tree, which can either be a leaf or an internal node.
- **Attributes**:
  - `is_leaf` (bool): To determine if the node is a leaf node.
  - `feature_index` (int): The feature index used to split.
  - `threshold` (float): The threshold value used to split the data.
  - `prediction` (int): The value of the prediction (if the node is a leaf).
  - `left` (dict): Left child node.
  - `right` (dict): Right child node.

In [21]:
def create_node(
    is_leaf: bool,
    feature_index: int = -1,
    threshold: float = -1,
    prediction: float = None,
) -> dict:
    """
    Creates a decision tree node.

    Args:
        is_leaf (bool): Whether the node is a leaf node.
        feature_index (int): The feature index used for splitting.
        threshold (float): The threshold value for splitting.
        prediction (float): The prediction value for leaf nodes.

    Returns:
        dict: A dictionary representing a decision tree node.
    """
    # fill code here

    return {
        "is_leaf": is_leaf,
        "feature_index": feature_index,
        "threshold": threshold,
        "prediction": prediction,
        "left": None,
        "right": None
    }

### **Splitting Dataset [4 points]**

- **Purpose**: Split the dataset into left and right parts based on a given feature and threshold.
-
  - Iterate through each instance in the dataset.
  - Compare the value of the selected feature with the given threshold.
  - Append the instance to either the left or right split based on whether the feature value is less than or equal to the threshold.
  - Make sure both the features and labels are split according

In [22]:
def split_dataset(
    df: pd.DataFrame, feature_index: str, threshold: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits the dataset based on a feature and a threshold.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        feature_index (str): The feature to split on.
        threshold (float): The threshold value to use for the split.

    Returns:
        tuple: A tuple containing the left and right datasets.
    """
    #fill code here
    left = df[df[feature_index] <= threshold]
    right = df[df[feature_index] > threshold]
    return left, right


### **Checking Purity [2 points]**

- **Purpose**: Determine if all the labels are the same.
-
  - If all labels are identical, the node should be considered pure, and it will become a leaf node.
  - Check if all elements are the same.
  - This function helps determine whether further splitting is necessary.

In [23]:
def is_pure(labels: pd.Series) -> bool:
    """
    Checks if all the labels are the same.

    Args:
        labels (pd.Series): The label values.

    Returns:
        bool: True if all labels are the same, False otherwise.
    """
    # fill code here
    return labels.nunique() == 1


### **Finding the Best Split [7 points]**

- **Purpose**: Find the best feature and threshold to split the dataset.
-
  - Iterate through each feature and determine possible thresholds.
  - Use entropy as the metric to evaluate the quality of the split.
  - For each threshold, calculate the entropy of the resulting splits and choose the one with the lowest entropy.
  - The best split will minimize the entropy, meaning the resulting splits are as pure as possible.


In [24]:
def find_best_split(
    df: pd.DataFrame, features: list[str], label_column: str
) -> tuple[str, float]:
    """
    Finds the best feature and threshold to split the dataset by minimizing entropy.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.

    Returns:
        tuple: The feature name and threshold value for the best split.
    """
    best_entropy = float("inf")
    best_feature = ""
    best_threshold = -1

    # fill code here

    for feature in features:
        thresholds = df[feature].unique()
        for threshold in thresholds:
            left, right = split_dataset(df, feature, threshold)
            if len(left) == 0 or len(right) == 0:
                continue

            entropy = calculate_entropy(left[label_column], right[label_column])
            if entropy < best_entropy:
                best_entropy = entropy
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

### **Calculating Entropy [7 points]**

- **Purpose**: Calculate the entropy for a given split of labels.
-
  - Entropy is a measure of impurity, where lower values indicate a purer split.
  - For each subset of labels (left and right), calculate the proportion of each class.
  - Use the formula for entropy
  - The entropy for the split is the weighted average of the left and right entropies.


In [25]:
def calculate_entropy(left_labels: pd.Series, right_labels: pd.Series) -> float:
    """
    Calculates the entropy for a split.

    Args:
        left_labels (pd.Series): The label values for the left split.
        right_labels (pd.Series): The label values for the right split.

    Returns:
        float: The entropy value for the split.
    """

    left_size = len(left_labels)
    right_size = len(right_labels)
    total_size = left_size + right_size

    if total_size == 0:
        return 0

    def entropy(labels):
        # fill code here
        counts = np.bincount(labels)
        probabilities = counts / len(labels)
        return -sum(p * np.log2(p) for p in probabilities if p > 0)

    # Calculate entropy for both left and right splits
    entropy_left = entropy(left_labels)
    entropy_right = entropy(right_labels)

    # Weighted average of both entropies
    entropy_left = entropy(left_labels)
    entropy_right = entropy(right_labels)

    # fill code here
    return (left_size / total_size) * entropy_left + (right_size / total_size) * entropy_right

### **Calculating Prediction [2 points]**

- **Purpose**: Calculate the prediction value for a leaf node (most common label).
- **Description**:
  - This function returns the most common label in the dataset, which is used as the prediction for a leaf node.


In [26]:
def calculate_prediction(labels: pd.Series) -> int:
    """
    Calculates the prediction value for a leaf node.

    Args:
        labels (pd.Series): The label values.

    Returns:
        int: The most common value of the labels.
    """
    # fill code here
    return labels.mode()[0]

### **Building the Decision Tree [7 points]**

- **Purpose**: Recursively build the decision tree by finding the best split and creating nodes.
- **Hints**:
  - Start with the current depth as 0 and increase it with each recursive call.
  - Stop the recursion if the current depth reaches the maximum depth or if the node is pure.
  - Use the `find_best_split` function to determine the best feature and threshold for splitting.
  - Create left and right child nodes recursively and attach them to the current node.

In [27]:
def build_tree(
    df: pd.DataFrame,
    features: list[str],
    label_column: str,
    current_depth: int,
    max_depth: int,
) -> dict:
    """
    Builds the decision tree recursively.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.
        current_depth (int): The current depth of the tree.
        max_depth (int): The maximum allowed depth of the tree.

    Returns:
        dict: The root node of the decision tree.
    """
    # fill code for base case
    labels = df[label_column]
    if is_pure(labels) or current_depth >= max_depth:
        prediction = calculate_prediction(labels)
        return create_node(is_leaf=True, prediction=prediction)


    # fill code to find best feature
    best_feature, best_threshold = find_best_split(df, features, label_column)

    # fill code to split the data
    if best_feature == "":
        prediction = calculate_prediction(labels)
        return create_node(is_leaf=True, prediction=prediction)

    left, right = split_dataset(df, best_feature, best_threshold)

    left_child = build_tree(left, features, label_column, current_depth + 1, max_depth)
    right_child = build_tree(right, features, label_column, current_depth + 1, max_depth)

    node = create_node(is_leaf=False, feature_index=best_feature, threshold=best_threshold)
    node["left"] = left_child
    node["right"] = right_child

    return node

### **Predicting Using the Tree [3 points]**

- **Purpose**: Predict the label for a given feature vector using the trained decision tree.
- **Hints**:
  - Start at the root of the tree and traverse down to a leaf node.
  - For each internal node, decide whether to move left or right based on the feature value and threshold.
  - When a leaf node is reached, return its prediction value.

In [28]:
def predict(tree: dict, sample: pd.Series) -> int:
    """
    Predicts the label for a given feature vector using the decision tree.

    Args:
        tree (dict): The root node of the decision tree.
        sample (pd.Series): The feature vector.

    Returns:
        int: The predicted label.
    """
    # fill code here

    if tree["is_leaf"]:
        return tree["prediction"]

    feature_value = sample[tree["feature_index"]]
    if feature_value <= tree["threshold"]:
        return predict(tree["left"], sample)
    else:
        return predict(tree["right"], sample)

### Using a Real-World Dataset and Comparison with Scikit-Learn

In [29]:
# Load the Iris dataset
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [30]:
# Prepare features and labels
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label_column = 'class'
iris['class'] = iris['class'].apply(lambda x: 0 if x == 'Iris-setosa' else (1 if x == 'Iris-versicolor' else 2))

In [31]:
# Split the dataset into training and testing
X_train, X_test = train_test_split(iris, test_size=0.2, random_state=42)

In [32]:
max_depth = 3
decision_tree = build_tree(X_train, features, label_column, 0, max_depth)

In [33]:
predictions = X_test.apply(lambda row: predict(decision_tree, row), axis=1)

In [34]:
custom_accuracy = accuracy_score(X_test[label_column], predictions)
print(f"Custom Decision Tree Accuracy: {custom_accuracy:.2f}")

Custom Decision Tree Accuracy: 1.00


In [35]:
clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
clf.fit(X_train[features], X_train[label_column])
sklearn_predictions = clf.predict(X_test[features])
sklearn_accuracy = accuracy_score(X_test[label_column], sklearn_predictions)
print(f"Scikit-Learn Decision Tree Accuracy: {sklearn_accuracy:.2f}")

Scikit-Learn Decision Tree Accuracy: 1.00


## Logistic Regression

### 1. Show that: [5 points]

$$
\sigma(-x) = 1 - \sigma(x)
$$

where
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

$$
\sigma(-x) = \frac{1}{1 + e^{x}}
$$

$$
1 - \sigma(x) = 1 - \frac{1}{1 + e^{-x}} = \frac{e^{-x}}{1 + e^{-x}} = \frac{\frac{1}{e^{x}}}{1 + \frac{1}{e^{x}}} = \frac{1}{1 + e^{x}} = \sigma(-x)
$$

### 2. Show that: [5 points]

$$
\frac{d}{dx}\sigma(x) = \sigma(x)(1 - \sigma(x))
$$


Given:

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

The derivative of $\sigma(x)$ with respect to $x$ is:

$$
\frac{d}{dx}\sigma(x) = \frac{d}{dx} \frac{1}{1 + e^{-x}} = \frac{e^{-x}}{(1 + e^{-x})^2}
$$

Now, substitute $\sigma(x) = \frac{1}{1 + e^{-x}}$:

$$
\frac{d}{dx}\sigma(x) = {\frac{1}{1 + e^{-x}}}{\frac{e^{-x}}{1 + e^{-x}}}  =  \sigma(x)(1 - \sigma(x))
$$


### 3. Gradient Descent [5 points]
You have a univariate function you wish to minimize, $f(w) = 5(w-11)^4$. Suppose you wish to perform gradient descent with constant step size $\alpha = 1/40$. Starting with $w_0 = 13$, perform 5 steps of gradient descent. What are $w_0, w_1, w_2, w_3, w_4, w_5$? What is the value of $f(w_5)$?

Given:

$f(w) = 5(w-11)^4$

$f'(w) = 20(w-11)^3 $

$w_{n+1} = w_n - \alpha f'(w_n)$

$w_0 = 13$

$$w_1 = w_0 - \alpha f'(0) = 13 - \frac{1}{40} (20 (13-11)^3) = 9 $$

$$w_2 = w_1 - \alpha f'(1) = 9 - \frac{1}{40} (20 (9-11)^3) = 13 $$

$$w_3 = w_2 - \alpha f'(2) = 13 - \frac{1}{40} (20 (13-11)^3) = 9 $$

$$w_4 = w_3 - \alpha f'(3) = 9 - \frac{1}{40} (20 (9-11)^3) = 13 $$

$$w_5 = w_4 - \alpha f'(5) = 13 - \frac{1}{40} (20 (13-11)^3) = 9 $$

$$f(w_5) = 5(9-11)^4 = 80$$